In [10]:
import random
import spacy
import nltk
from goose3 import Goose
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import scrolledtext
from PIL import Image, ImageTk
import io
from deep_translator import GoogleTranslator

In [11]:
nltk.download('punkt_tab')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt_tab to /home/muril/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/muril/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
!python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

welcome_words_input = ['olá', 'oi', 'hey', 'bom dia', 'boa tarde', 'boa noite']
welcome_words_output = [
    'Olá! Como posso ajudar você sobre Hobbit hoje?',
    'Olá! Fique à vontade para perguntar sobre Hobbit.',
    'Oi! Em que posso ajudar com relação à Hobbit?'
]

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.8.0/pt_core_news_sm-3.8.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [13]:
# Função para mensagens de boas-vindas
def welcome_message(text):
    for word in text.lower().split():
        if word in welcome_words_input:
            return random.choice(welcome_words_output)
    return None

# Função de pré-processamento de texto
def preprocessing(sentence):
    sentence = sentence.lower()
    tokens = [token.text for token in nlp(sentence) if not (token.is_stop or token.like_num
                                                              or token.is_punct or token.is_space
                                                              or len(token) == 1)]
    return ' '.join(tokens)

In [14]:
# Coletando e processando as sentenças
original_sentences = []

g = Goose()
article = g.extract(url='https://terramedia.fandom.com/wiki/O_Hobbit')

sentences = nltk.sent_tokenize(article.cleaned_text)
original_sentences.extend(sentences)

In [15]:
def traduzir_texto(texto):
    return GoogleTranslator(source='auto', target='en').translate(texto)

In [16]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Inicializar o analisador
sia = SentimentIntensityAnalyzer()

# Função para análise de sentimento
def analisar_sentimento(texto):
    
    traduzido = traduzir_texto(texto)

    scores = sia.polarity_scores(traduzido)
    sentimento = "neutro"
    if scores['compound'] > 0.05:
        sentimento = "positivo"
    elif scores['compound'] < -0.05:
        sentimento = "negativo"
    return sentimento, scores

In [17]:
# Pré-processamento das sentenças
cleaned_sentences = [preprocessing(sentence) for sentence in original_sentences]

# Função para responder às perguntas do usuário
def answer(user_text, threshold=0.2):
    user_text_clean = preprocessing(user_text)
    all_sentences = cleaned_sentences + [user_text_clean]
    
    good_answers = [
        "Que bom ouvir isso! 😊 Espero que sua experiência continue positiva.",
        "Adoro essa energia! Se precisar de algo, estou aqui.",
        "Ótimo saber disso! Vamos seguir em frente.",
        "Fico feliz por você compartilhar isso. Conte comigo!",
        "Que legal! Sempre bom ver algo positivo acontecendo.",
        "Isso é incrível! Vamos manter esse clima bom.",
        "Que bom saber disso! Estou aqui para continuar ajudando.",
        "Adoro essa vibe! O que mais posso fazer por você?",
        "Fico contente com isso! Pode contar comigo para o que precisar.",
        "Muito bom ouvir isso! Vamos juntos nessa."
    ]
    
    bad_answers = [
        "Sinto muito que as coisas estejam assim. Estou aqui para ajudar.",
        "Parece que algo não está bem. Se precisar, pode contar comigo.",
        "Lamento que você esteja enfrentando isso. Posso ajudar de alguma forma?",
        "Entendo que não seja fácil. Se precisar desabafar, estou aqui.",
        "Sinto que há algo errado. Vamos tentar resolver juntos?",
        "Lamento que esteja se sentindo assim. Estou aqui para ajudar no que puder.",
        "Entendi sua frustração. Se houver algo que eu possa fazer, me avise.",
        "Parece que as coisas estão difíceis. Estou aqui para ajudar no que precisar.",
        "Compreendo sua situação. Fique à vontade para me contar mais.",
        "Lamento por isso. Estou por aqui para ajudar como puder."
    ]
    
    feeling, score = analisar_sentimento(user_text)
    print(f"Sentimento: {feeling}")
    
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(all_sentences)
    
    similarity = cosine_similarity(tfidf[-1], tfidf)
    similarity_scores = similarity.flatten()
    
    # Encontrar a sentença com maior similaridade, excluindo a própria
    highest_idx = similarity_scores.argsort()[-2]
    highest_score = similarity_scores[highest_idx]
    
    sentimento = ""
    if feeling == "positivo":
        sentimento = good_answers[random.randint(1,10)]
    elif feeling == "negativo":
        sentimento = bad_answers[random.randint(1,10)]
    
    if highest_score < threshold:
        frase = 'Desculpe, não encontrei uma resposta adequada para sua pergunta.'
    else:        
        frase = original_sentences[highest_idx], 
    
    return frase, sentimento

# Função para gerar a nuvem de palavras
def generate_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    return wordcloud

In [ ]:
# Preparando a base de conhecimento para a nuvem de palavras
full_text = ' '.join(cleaned_sentences)
wordcloud = generate_wordcloud(full_text)

# Salvando a nuvem de palavras em memória
image_stream = io.BytesIO()
wordcloud.to_image().save(image_stream, format='PNG')
image_stream.seek(0)
wordcloud_image = Image.open(image_stream)

# Função do Chatbot para a interface
def send_message():
    user_input = user_entry.get()
    if user_input.lower() in ['sair', 'quit', 'exit']:
        chat_window.config(state=NORMAL)
        chat_window.insert(END, "Chatbot: Até logo!\n")
        chat_window.config(state=DISABLED)
        root.after(2000, root.destroy)
        return
    if user_input.strip() == '':
        return
    chat_window.config(state=NORMAL)
    chat_window.insert(END, f"Você: {user_input}\n")
    user_entry.delete(0, END)
    
    welcome = welcome_message(user_input)
    if welcome:
        response = welcome
    else:
        response, feeling = answer(user_input)
        
    if feeling != '':
        chat_window.insert(END, f"Chatbot: {feeling}\n")
        
    chat_window.insert(END, f"Chatbot: {response}\n")
    chat_window.yview(END)
    chat_window.config(state=DISABLED)

# Função para limpar o chat
def clear_chat():
    chat_window.config(state=NORMAL)
    chat_window.delete('1.0', END)  # Remove todo o texto da área de chat
    chat_window.insert(END, "Chatbot: Olá! Eu sou um chatbot. Faça sua pergunta ou digite 'sair' para encerrar.\n")
    chat_window.config(state=DISABLED)
    

# Inicializando a janela do Tkinter
root = Tk()
root.title("Chatbot: O Hobbit")
root.geometry("800x825")
root.resizable(False, False)

# Frame para o chat
chat_frame = Frame(root)
chat_frame.pack(pady=10)

# Área de texto para exibir o chat
chat_window = scrolledtext.ScrolledText(chat_frame, wrap=WORD, width=80, height=20, state=DISABLED)
chat_window.pack()

# Frame para a entrada do usuário
input_frame = Frame(root)
input_frame.pack(pady=10)

# Entrada de texto para o usuário
user_entry = Entry(input_frame, width=70)
user_entry.pack(side='left', padx=10)

# Botão para enviar a mensagem
send_button = Button(input_frame, text="Enviar", command=send_message)
send_button.pack(side='left')

send_button.pack(side='left', padx=10)

# Botão para limpar o chat
clear_button = Button(input_frame, text="Limpar", command=clear_chat)
clear_button.pack(side='left')

# Frame para a nuvem de palavras
wordcloud_frame = Frame(root)
wordcloud_frame.pack(pady=10)

# Label para a nuvem de palavras
wordcloud_label = Label(wordcloud_frame, borderwidth=2, relief="solid")
wordcloud_label.pack()  

# Função para exibir a nuvem de palavras
def display_wordcloud():
    resized_image = wordcloud_image.resize((750, 375), Image.Resampling.LANCZOS)
    tk_image = ImageTk.PhotoImage(resized_image)
    wordcloud_label.configure(image=tk_image)
    wordcloud_label.image = tk_image

# Exibindo a nuvem de palavras ao iniciar
display_wordcloud()

# Iniciando o Chatbot
def start_chatbot():
    chat_window.config(state=NORMAL)
    chat_window.insert(END, "Chatbot: Olá! Eu sou um chatbot. Faça sua pergunta ou digite 'sair' para encerrar.\n")
    chat_window.config(state=DISABLED)

start_chatbot()

# Bind Enter key to send message
root.bind('<Return>', lambda event: send_message())
    
# Executando a interface
root.mainloop()


Sentimento: negativo
Sentimento: negativo
Sentimento: positivo
Sentimento: positivo
Sentimento: neutro
Sentimento: positivo
Sentimento: negativo
Sentimento: neutro
Sentimento: positivo
Sentimento: neutro
Sentimento: positivo
Sentimento: positivo
Sentimento: positivo
Sentimento: positivo
Sentimento: negativo
Sentimento: neutro
Sentimento: negativo
Sentimento: negativo
Sentimento: positivo
